In [1]:
!pip install arch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 916.4/916.4 kB 15.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import yfinance as yf
import arch

In [3]:
from google.colab import drive
drive.mount('/content/drive')
import os

Mounted at /content/drive


In [4]:
os.chdir('/content/drive/My Drive/thesis/3rd_presentation')
df = pd.read_csv('non_nan_4.csv')

In [5]:
df

,Date,unemployment_rate,CPI,treasury_yield,GDP_growth,SP500_return,AZN,BMY,JNJ,LLY,MRK,NVO,NVS,PFE,ROG,inflation_change,unemp_change,treasury_yield_change
0,2000-02-01,4.1,170.000,6.661000,0.496560,-1.522563,-12.828964,-13.228004,-16.339821,-11.121498,-21.701151,2.220031,3.838386,-11.226228,54.440789,1.000,-0.1,-0.141500
1,2000-03-01,4.0,171.000,6.519500,0.511258,9.413333,22.264136,-0.218329,-2.079067,5.804243,0.913712,8.390897,6.420237,14.101954,6.922258,1.000,-0.1,-0.141500
2,2000-04-01,3.8,170.900,6.256522,1.327803,-3.266805,5.567379,-8.205683,17.437698,23.153694,12.400712,-0.097663,2.559423,15.213674,7.370518,-0.100,-0.2,-0.262978
3,2000-05-01,4.0,171.200,5.990526,-0.181797,-1.572223,-0.148357,5.395746,8.484832,-1.296597,7.374072,20.863985,5.169310,5.638019,-8.163265,0.300,0.2,-0.265995
4,2000-06-01,4.0,172.200,6.440455,0.305565,1.728613,10.549735,5.788826,14.239888,31.641749,3.078671,2.813690,8.474599,8.076012,13.131313,1.000,0.0,0.449928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,2023-01-01,3.4,300.536,3.616190,0.390254,6.776820,-3.584079,0.972908,-7.489384,-5.928822,-2.549213,2.541749,-0.110227,-13.817335,16.968326,1.546,-0.1,-0.274810
276,2023-02-01,3.6,301.648,3.531500,-0.442183,-2.514271,-0.290649,-4.328217,-6.217115,-9.568502,-1.089288,1.592445,-7.172811,-7.286115,5.451681,1.112,0.2,-0.084690
277,2023-03-01,3.5,301.808,3.746842,-0.442183,3.313488,8.035329,0.507544,1.862736,10.703390,0.141189,12.873250,9.367574,0.566924,11.025813,0.160,-0.1,0.215342
278,2023-04-01,3.4,302.918,3.663043,-0.442183,1.985238,5.489119,-3.664707,5.612908,15.269915,9.289214,5.836894,16.334413,-4.681371,-1.517467,1.110,-0.1,-0.083799


In [6]:
def create_lags(data, tickers, covs, period):
    lagged_covs = []  # List to store the names of lagged columns

    for covariate in covs:
        for i in period:
            lagged_column_name = f'{covariate}_lag{i}'
            data[lagged_column_name] = data[covariate].shift(i)
            lagged_covs.append(lagged_column_name)

    lagged_tickers = []  # List to store the names of lagged columns

    for tick in tickers:
        for i in period:
            lagged_column_name = f'{tick}_lag{i}'
            data[lagged_column_name] = data[tick].shift(i)
            lagged_tickers.append(lagged_column_name)

    data.dropna(inplace=True)
    data = data.reset_index(drop=True)

    return data, lagged_covs, lagged_tickers

In [7]:
tickers = ['AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO', 'NVS', 'PFE','ROG']
covs = ['unemployment_rate', 'CPI', 'treasury_yield', 'GDP_growth', 'SP500_return', 'inflation_change', 'unemp_change', 'treasury_yield_change']
period = [1, 2, 3]

data, lagged_covs, lagged_tickers = create_lags(df, tickers, covs, period)

In [8]:
df

,Date,unemployment_rate,CPI,treasury_yield,GDP_growth,SP500_return,AZN,BMY,JNJ,LLY,...,NVO_lag3,NVS_lag1,NVS_lag2,NVS_lag3,PFE_lag1,PFE_lag2,PFE_lag3,ROG_lag1,ROG_lag2,ROG_lag3
3,2000-05-01,4.0,171.200,5.990526,-0.181797,-1.572223,-0.148357,5.395746,8.484832,-1.296597,...,2.220031,2.559423,6.420237,3.838386,15.213674,14.101954,-11.226228,7.370518,6.922258,54.440789
4,2000-06-01,4.0,172.200,6.440455,0.305565,1.728613,10.549735,5.788826,14.239888,31.641749,...,8.390897,5.169310,2.559423,6.420237,5.638019,15.213674,14.101954,-8.163265,7.370518,6.922258
5,2000-07-01,4.0,172.700,6.097273,-0.653855,-1.338453,-8.064479,-11.373392,-8.650271,4.005041,...,-0.097663,8.474599,5.169310,2.559423,8.076012,5.638019,15.213674,13.131313,-8.163265,7.370518
6,2000-08-01,4.1,172.700,6.054000,0.795710,6.534124,6.578935,3.131245,-1.192113,-29.708189,...,20.863985,-3.593737,8.474599,5.169310,-9.635421,8.076012,5.638019,2.142857,13.131313,-8.163265
7,2000-09-01,3.9,173.600,5.826087,0.249670,-5.723123,15.363525,8.458631,2.496165,11.482185,...,2.813690,-1.944901,-3.593737,8.474599,-0.540360,-9.635421,8.076012,-2.447552,2.142857,13.131313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,2023-01-01,3.4,300.536,3.616190,0.390254,6.776820,-3.584079,0.972908,-7.489384,-5.928822,...,9.244201,1.295221,10.390734,6.735944,3.090302,7.690670,6.375682,9.446071,-53.665066,-2.707956
276,2023-02-01,3.6,301.648,3.531500,-0.442183,-2.514271,-0.290649,-4.328217,-6.217115,-9.568502,...,14.479974,-0.110227,1.295221,10.390734,-13.817335,3.090302,7.690670,16.968326,9.446071,-53.665066
277,2023-03-01,3.5,301.808,3.746842,-0.442183,3.313488,8.035329,0.507544,1.862736,10.703390,...,8.619586,-7.172811,-0.110227,1.295221,-7.286115,-13.817335,3.090302,5.451681,16.968326,9.446071
278,2023-04-01,3.4,302.918,3.663043,-0.442183,1.985238,5.489119,-3.664707,5.612908,15.269915,...,2.541749,9.367574,-7.172811,-0.110227,0.566924,-7.286115,-13.817335,11.025813,5.451681,16.968326


In [ ]:
"""def tick_loss(alpha, returns, var):
    df = pd.DataFrame({'Return': returns, 'VaR': var})
    df['Indicator'] = np.where(df['Return'] < df['VaR'], 1, 0)

    t_loss = 0

    for i in df.index:
        t_loss += (
            alpha * (df['Return'][i] - df['VaR'][i]) * (1 - df['Indicator'][i])
            + (1 - alpha) * (df['VaR'][i] - df['Return'][i]) * df['Indicator'][i]
        )

    return t_loss, df"""

In [9]:
def tick_loss(alpha, returns, var):
    df = pd.DataFrame({'Return': returns, 'VaR': var})
    df['Indicator'] = np.where(df['Return'] < df['VaR'], 1, 0)

    t_loss = 0

    for i in df.index:
        t_loss += (
            alpha * (df['Return'][i] - df['VaR'][i]) * (1 - df['Indicator'][i])
            + (1 - alpha) * (df['VaR'][i] - df['Return'][i]) * df['Indicator'][i]
        )

    return t_loss

# 1 MONTH AHEAD:

In [13]:
import pandas as pd
import statsmodels.api as sm
from arch import arch_model
from scipy.stats import norm

def rolling_1m_forecast(data, dates, ticker, alpha):
    t_loss = 0
    dta = data.copy()

    for d in dates:
        train_df = dta[dta.Date <= d]
        test_df = dta[dta.Date > d]
        test_df = test_df.reset_index(drop=True)

        X_train = train_df.drop(columns=[ticker, 'Date'])
        y_train = train_df[ticker]

        X_test = test_df.drop(columns=[ticker, 'Date'])
        X_test = X_test.iloc[0]
        y_test = test_df[ticker]

        garch_model = arch_model(y_train, vol='Garch', p=1, q=1)
        garch_results = garch_model.fit()

        forecasts = garch_results.forecast(start=0, horizon=1)
        cond_vol_forecast = forecasts.variance.iloc[-1].apply(lambda x: np.sqrt(x))
        mean_forecast = forecasts.mean.iloc[-1]

        VaR_forecast = mean_forecast + cond_vol_forecast * norm.ppf(alpha)

        loss = tick_loss(alpha, y_test.iloc[0], VaR_forecast)

        t_loss += loss

    t_loss = t_loss / 12

    return t_loss

In [11]:
dates1m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01', '2022-12-01', '2023-01-01', '2023-02-01', '2023-03-01', '2023-04-01']

In [14]:
for t in tickers:
    loss = rolling_1m_forecast(data, dates1m, t, 0.05)
    print(f'{t}: {loss}')

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1212.2397444537628
Iteration:      2,   Func. Count:     13,   Neg. LLF: 878.8800633143923
Iteration:      3,   Func. Count:     19,   Neg. LLF: 876.5126927333897
Iteration:      4,   Func. Count:     26,   Neg. LLF: 874.6973681636873
Iteration:      5,   Func. Count:     31,   Neg. LLF: 874.6903544449096
Iteration:      6,   Func. Count:     36,   Neg. LLF: 874.6560717780487
Iteration:      7,   Func. Count:     41,   Neg. LLF: 874.6348053303011
Iteration:      8,   Func. Count:     46,   Neg. LLF: 874.6216452705775
Iteration:      9,   Func. Count:     51,   Neg. LLF: 874.6199930891166
Iteration:     10,   Func. Count:     56,   Neg. LLF: 874.6199821054033
Iteration:     11,   Func. Count:     60,   Neg. LLF: 874.6199821052801
Optimization terminated successfully    (Exit mode 0)
            Current function value: 874.6199821054033
            Iterations: 11
            Function evaluations: 60
            Gradient evaluations: 1

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      4,   Func. Count:     26,   Neg. LLF: 888.2890739635261
Iteration:      5,   Func. Count:     31,   Neg. LLF: 888.2857287199915
Iteration:      6,   Func. Count:     36,   Neg. LLF: 888.2700531031528
Iteration:      7,   Func. Count:     41,   Neg. LLF: 888.235257508208
Iteration:      8,   Func. Count:     46,   Neg. LLF: 888.2348394809553
Iteration:      9,   Func. Count:     51,   Neg. LLF: 888.2348350492899
Iteration:     10,   Func. Count:     55,   Neg. LLF: 888.2348350491477
Optimization terminated successfully    (Exit mode 0)
            Current function value: 888.2348350492899
            Iterations: 10
            Function evaluations: 55
            Gradient evaluations: 10
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1237.7876645235606
Iteration:      2,   Func. Count:     13,   Neg. LLF: 895.3056383954452
Iteration:      3,   Func. Count:     19,   Neg. LLF: 892.1749916436477
Iteration:      4,   Func. Count:     26,   Neg. LLF: 891.5621526471309

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1253.9328953167324
Iteration:      2,   Func. Count:     13,   Neg. LLF: 906.7418070698182
Iteration:      3,   Func. Count:     19,   Neg. LLF: 904.8008315732066
Iteration:      4,   Func. Count:     26,   Neg. LLF: 902.7254979210031
Iteration:      5,   Func. Count:     31,   Neg. LLF: 902.7226521425612
Iteration:      6,   Func. Count:     36,   Neg. LLF: 902.7078848604058
Iteration:      7,   Func. Count:     41,   Neg. LLF: 902.6810797105925
Iteration:      8,   Func. Count:     46,   Neg. LLF: 902.679903213058
Iteration:      9,   Func. Count:     51,   Neg. LLF: 902.6798556039007
Iteration:     10,   Func. Count:     56,   Neg. LLF: 902.6798547282827
Optimization terminated successfully    (Exit mode 0)
            Current function value: 902.6798547282827
            Iterations: 10
            Function evaluations: 56
            Gradient evaluations: 10
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1258.496015236461

...

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      8,   Func. Count:     49,   Neg. LLF: 1052.1832724647165
Iteration:      9,   Func. Count:     55,   Neg. LLF: 1052.0435153380367
Iteration:     10,   Func. Count:     60,   Neg. LLF: 1052.0390347801442
Iteration:     11,   Func. Count:     65,   Neg. LLF: 1052.0387828195512
Iteration:     12,   Func. Count:     70,   Neg. LLF: 1052.0387816594484
Iteration:     13,   Func. Count:     75,   Neg. LLF: 1052.0387800035776
Iteration:     14,   Func. Count:     79,   Neg. LLF: 1052.0387800078834
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1052.0387800035776
            Iterations: 14
            Function evaluations: 79
            Gradient evaluations: 14
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1173.5312981645138
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1103.5635157728589
Iteration:      3,   Func. Count:     20,   Neg. LLF: 1056.8225089020862
Iteration:      4,   Func. Count:     25,   Neg. LLF: 1088.9

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


Iteration:      9,   Func. Count:     55,   Neg. LLF: 1059.8442718792248
Iteration:     10,   Func. Count:     61,   Neg. LLF: 1059.8201149528445
Iteration:     11,   Func. Count:     66,   Neg. LLF: 1059.81924365574
Iteration:     12,   Func. Count:     71,   Neg. LLF: 1059.8192085030832
Iteration:     13,   Func. Count:     76,   Neg. LLF: 1059.8191918563493
Iteration:     14,   Func. Count:     81,   Neg. LLF: 1059.8191707319822
Iteration:     15,   Func. Count:     86,   Neg. LLF: 1059.8191690796416
Iteration:     16,   Func. Count:     91,   Neg. LLF: 4544.99073059276
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1059.8191690761073
            Iterations: 17
            Function evaluations: 97
            Gradient evaluations: 16
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1182.042973451109
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1128.2198746474905
Iteration:      3,   Func. Count:     20,   Neg. LLF: 1064.141923

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


# 2 MONTHS AHEAD:

In [15]:
import pandas as pd
import statsmodels.api as sm
from arch import arch_model
from scipy.stats import norm

def rolling_3m_forecast(data, dates, ticker, alpha):
    t_loss = 0
    dta = data.copy()

    for d in dates:
        train_df = dta[dta.Date <= d]
        test_df = dta[dta.Date > d]
        test_df = test_df.reset_index(drop=True)

        X_train = train_df.drop(columns=[ticker, 'Date'])
        y_train = train_df[ticker]

        X_test = test_df.drop(columns=[ticker, 'Date'])
        X_test = X_test.iloc[0]
        y_test = test_df[ticker]

        garch_model = arch_model(y_train, vol='Garch', p=1, q=1)
        garch_results = garch_model.fit()

        forecasts = garch_results.forecast(start=0, horizon=3)  # Forecast for 3 months ahead
        cond_vol_forecast = forecasts.variance.iloc[-1].apply(lambda x: np.sqrt(x))
        mean_forecast = forecasts.mean.iloc[-1]

        VaR_forecast = mean_forecast + cond_vol_forecast * norm.ppf(alpha)

        loss = tick_loss(alpha, y_test.iloc[0], VaR_forecast)

        t_loss += loss

    t_loss = t_loss / 10  # Adjusted for 10 months (since 'dates3m' has 10 elements)

    return t_loss

In [16]:
dates3m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01', '2022-12-01', '2023-01-01', '2023-02-01']

In [17]:
for t in tickers:
    loss = rolling_3m_forecast(data, dates3m, t, 0.05)
    print(f'{t}: {loss}')

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1212.2397444537628
Iteration:      2,   Func. Count:     13,   Neg. LLF: 878.8800633143923
Iteration:      3,   Func. Count:     19,   Neg. LLF: 876.5126927333897
Iteration:      4,   Func. Count:     26,   Neg. LLF: 874.6973681636873
Iteration:      5,   Func. Count:     31,   Neg. LLF: 874.6903544449096
Iteration:      6,   Func. Count:     36,   Neg. LLF: 874.6560717780487
Iteration:      7,   Func. Count:     41,   Neg. LLF: 874.6348053303011
Iteration:      8,   Func. Count:     46,   Neg. LLF: 874.6216452705775
Iteration:      9,   Func. Count:     51,   Neg. LLF: 874.6199930891166
Iteration:     10,   Func. Count:     56,   Neg. LLF: 874.6199821054033
Iteration:     11,   Func. Count:     60,   Neg. LLF: 874.6199821052801
Optimization terminated successfully    (Exit mode 0)
            Current function value: 874.6199821054033
            Iterations: 11
            Function evaluations: 60
            Gradient evaluations: 1

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1232.8412629115683
Iteration:      2,   Func. Count:     13,   Neg. LLF: 892.5151785751364
Iteration:      3,   Func. Count:     19,   Neg. LLF: 889.8880646630623
Iteration:      4,   Func. Count:     26,   Neg. LLF: 888.2890739635261
Iteration:      5,   Func. Count:     31,   Neg. LLF: 888.2857287199915
Iteration:      6,   Func. Count:     36,   Neg. LLF: 888.2700531031528
Iteration:      7,   Func. Count:     41,   Neg. LLF: 888.235257508208
Iteration:      8,   Func. Count:     46,   Neg. LLF: 888.2348394809553
Iteration:      9,   Func. Count:     51,   Neg. LLF: 888.2348350492899
Iteration:     10,   Func. Count:     55,   Neg. LLF: 888.2348350491477
Optimization terminated successfully    (Exit mode 0)
            Current function value: 888.2348350492899
            Iterations: 10
            Function evaluations: 55
            Gradient evaluations: 10
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1237.787664523560

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1253.9328953167324
Iteration:      2,   Func. Count:     13,   Neg. LLF: 906.7418070698182
Iteration:      3,   Func. Count:     19,   Neg. LLF: 904.8008315732066
Iteration:      4,   Func. Count:     26,   Neg. LLF: 902.7254979210031
Iteration:      5,   Func. Count:     31,   Neg. LLF: 902.7226521425612
Iteration:      6,   Func. Count:     36,   Neg. LLF: 902.7078848604058
Iteration:      7,   Func. Count:     41,   Neg. LLF: 902.6810797105925
Iteration:      8,   Func. Count:     46,   Neg. LLF: 902.679903213058
Iteration:      9,   Func. Count:     51,   Neg. LLF: 902.6798556039007
Iteration:     10,   Func. Count:     56,   Neg. LLF: 902.6798547282827
Optimization terminated successfully    (Exit mode 0)
            Current function value: 902.6798547282827
            Iterations: 10
            Function evaluations: 56
            Gradient evaluations: 10
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1258.496015236461

...

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:     10,   Func. Count:     55,   Neg. LLF: 1034.2630812202572
Iteration:     11,   Func. Count:     60,   Neg. LLF: 1034.2627809298356
Iteration:     12,   Func. Count:     65,   Neg. LLF: 1034.2627477455087
Iteration:     13,   Func. Count:     70,   Neg. LLF: 1034.262745030633
Iteration:     14,   Func. Count:     75,   Neg. LLF: 1034.2627446806737
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1034.2627446806737
            Iterations: 14
            Function evaluations: 75
            Gradient evaluations: 14
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1164.686867218073
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1067.8405488378116
Iteration:      3,   Func. Count:     20,   Neg. LLF: 1048.887969604437
Iteration:      4,   Func. Count:     25,   Neg. LLF: 1079.8863752982816
Iteration:      5,   Func. Count:     31,   Neg. LLF: 1051.113267207147
Iteration:      6,   Func. Count:     37,   Neg. LLF: 1049.16397

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1173.5312981645138
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1103.5635157728589
Iteration:      3,   Func. Count:     20,   Neg. LLF: 1056.8225089020862
Iteration:      4,   Func. Count:     25,   Neg. LLF: 1088.934887933981
Iteration:      5,   Func. Count:     31,   Neg. LLF: 1059.5762698399049
Iteration:      6,   Func. Count:     37,   Neg. LLF: 1057.324466999094
Iteration:      7,   Func. Count:     43,   Neg. LLF: 1056.5996080422344
Iteration:      8,   Func. Count:     49,   Neg. LLF: 1056.2996076429122
Iteration:      9,   Func. Count:     55,   Neg. LLF: 1056.252073735367
Iteration:     10,   Func. Count:     60,   Neg. LLF: 1056.2501190541698
Iteration:     11,   Func. Count:     65,   Neg. LLF: 1056.249650719259
Iteration:     12,   Func. Count:     70,   Neg. LLF: 1056.2495817996708
Iteration:     13,   Func. Count:     75,   Neg. LLF: 1056.2495640248285
Iteration:     14,   Func. Count:     79,   Neg. LLF: 1

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


# 6 MONTHS AHEAD:

In [18]:
import pandas as pd
import statsmodels.api as sm
from arch import arch_model
from scipy.stats import norm

def rolling_6m_forecast(data, dates, ticker, alpha):
    t_loss = 0
    dta = data.copy()

    for d in dates:
        train_df = dta[dta.Date <= d]
        test_df = dta[dta.Date > d]
        test_df = test_df.reset_index(drop=True)

        X_train = train_df.drop(columns=[ticker, 'Date'])
        y_train = train_df[ticker]

        X_test = test_df.drop(columns=[ticker, 'Date'])
        X_test = X_test.iloc[0]
        y_test = test_df[ticker]

        garch_model = arch_model(y_train, vol='Garch', p=1, q=1)
        garch_results = garch_model.fit()

        forecasts = garch_results.forecast(start=0, horizon=6)  # Forecast for 6 months ahead
        cond_vol_forecast = forecasts.variance.iloc[-1].apply(lambda x: np.sqrt(x))
        mean_forecast = forecasts.mean.iloc[-1]

        VaR_forecast = mean_forecast + cond_vol_forecast * norm.ppf(alpha)

        loss = tick_loss(alpha, y_test.iloc[0], VaR_forecast)

        t_loss += loss

    t_loss = t_loss / 7  # Adjusted for 7 months (since 'dates6m' has 7 elements)

    return t_loss

In [19]:
dates6m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01']

In [20]:
for t in tickers:
    loss = rolling_6m_forecast(data, dates6m, t, 0.05)
    print(f'{t}: {loss}')

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1212.2397444537628
Iteration:      2,   Func. Count:     13,   Neg. LLF: 878.8800633143923
Iteration:      3,   Func. Count:     19,   Neg. LLF: 876.5126927333897
Iteration:      4,   Func. Count:     26,   Neg. LLF: 874.6973681636873
Iteration:      5,   Func. Count:     31,   Neg. LLF: 874.6903544449096
Iteration:      6,   Func. Count:     36,   Neg. LLF: 874.6560717780487
Iteration:      7,   Func. Count:     41,   Neg. LLF: 874.6348053303011
Iteration:      8,   Func. Count:     46,   Neg. LLF: 874.6216452705775
Iteration:      9,   Func. Count:     51,   Neg. LLF: 874.6199930891166
Iteration:     10,   Func. Count:     56,   Neg. LLF: 874.6199821054033
Iteration:     11,   Func. Count:     60,   Neg. LLF: 874.6199821052801
Optimization terminated successfully    (Exit mode 0)
            Current function value: 874.6199821054033
            Iterations: 11
            Function evaluations: 60
            Gradient evaluations: 1

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter


Iteration:      8,   Func. Count:     46,   Neg. LLF: 883.4737747768788
Iteration:      9,   Func. Count:     51,   Neg. LLF: 883.4696451596949
Iteration:     10,   Func. Count:     56,   Neg. LLF: 883.4696170540337
Iteration:     11,   Func. Count:     60,   Neg. LLF: 883.4696170542519
Optimization terminated successfully    (Exit mode 0)
            Current function value: 883.4696170540337
            Iterations: 11
            Function evaluations: 60
            Gradient evaluations: 11
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1232.8412629115683
Iteration:      2,   Func. Count:     13,   Neg. LLF: 892.5151785751364
Iteration:      3,   Func. Count:     19,   Neg. LLF: 889.8880646630623
Iteration:      4,   Func. Count:     26,   Neg. LLF: 888.2890739635261
Iteration:      5,   Func. Count:     31,   Neg. LLF: 888.2857287199915
Iteration:      6,   Func. Count:     36,   Neg. LLF: 888.2700531031528
Iteration:      7,   Func. Count:     41,   Neg. LLF: 888.23525750820

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      7,   Func. Count:     42,   Neg. LLF: 896.59208224076
Iteration:      8,   Func. Count:     47,   Neg. LLF: 896.587190477322
Iteration:      9,   Func. Count:     52,   Neg. LLF: 896.5869081446032
Iteration:     10,   Func. Count:     57,   Neg. LLF: 896.5869067461391
Iteration:     11,   Func. Count:     61,   Neg. LLF: 896.5869067461389
Optimization terminated successfully    (Exit mode 0)
            Current function value: 896.5869067461391
            Iterations: 11
            Function evaluations: 61
            Gradient evaluations: 11
AZN: 4.897926939413389
Iteration:      1,   Func. Count:      6,   Neg. LLF: 910.8761979217478
Iteration:      2,   Func. Count:     13,   Neg. LLF: 912.7565471745722
Iteration:      3,   Func. Count:     20,   Neg. LLF: 913.0722689990389
Iteration:      4,   Func. Count:     27,   Neg. LLF: 891.7564018429989
Iteration:      5,   Func. Count:     32,   Neg. LLF: 891.7563027328458
Iteration:      6,   Func. Count:     37,   Neg. LL

...

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

PFE: 3.4538844018070227
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1221.0166728153697
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1019.1644029800323
Iteration:      3,   Func. Count:     19,   Neg. LLF: 1017.3061587014577
Iteration:      4,   Func. Count:     24,   Neg. LLF: 1017.3060349893901
Iteration:      5,   Func. Count:     29,   Neg. LLF: 1017.305916637994
Iteration:      6,   Func. Count:     34,   Neg. LLF: 1017.3053485200937
Iteration:      7,   Func. Count:     39,   Neg. LLF: 1017.3038504275205
Iteration:      8,   Func. Count:     44,   Neg. LLF: 1017.3017595917572
Iteration:      9,   Func. Count:     49,   Neg. LLF: 1017.3000463559385
Iteration:     10,   Func. Count:     54,   Neg. LLF: 1017.2993675428173
Iteration:     11,   Func. Count:     59,   Neg. LLF: 1017.2993362375026
Iteration:     12,   Func. Count:     64,   Neg. LLF: 1017.2993351230318
Iteration:     13,   Func. Count:     68,   Neg. LLF: 1017.2993351232732
Optimization terminated succ

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1244.0097929089118
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1036.41950801081
Iteration:      3,   Func. Count:     19,   Neg. LLF: 1034.2654744756092
Iteration:      4,   Func. Count:     24,   Neg. LLF: 1034.265484439243
Iteration:      5,   Func. Count:     30,   Neg. LLF: 1034.2653661958934
Iteration:      6,   Func. Count:     35,   Neg. LLF: 1034.2652887343463
Iteration:      7,   Func. Count:     40,   Neg. LLF: 1034.2650645363185
Iteration:      8,   Func. Count:     45,   Neg. LLF: 1034.2646220372608
Iteration:      9,   Func. Count:     50,   Neg. LLF: 1034.2638128195733
Iteration:     10,   Func. Count:     55,   Neg. LLF: 1034.2630812202572
Iteration:     11,   Func. Count:     60,   Neg. LLF: 1034.2627809298356
Iteration:     12,   Func. Count:     65,   Neg. LLF: 1034.2627477455087
Iteration:     13,   Func. Count:     70,   Neg. LLF: 1034.262745030633
Iteration:     14,   Func. Count:     75,   Neg. LLF: 1

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


# 9 MONTHS AHEAD:

In [21]:
import pandas as pd
import statsmodels.api as sm
from arch import arch_model
from scipy.stats import norm

def rolling_9m_forecast(data, dates, ticker, alpha):
    t_loss = 0
    dta = data.copy()

    for d in dates:
        train_df = dta[dta.Date <= d]
        test_df = dta[dta.Date > d]
        test_df = test_df.reset_index(drop=True)

        X_train = train_df.drop(columns=[ticker, 'Date'])
        y_train = train_df[ticker]

        X_test = test_df.drop(columns=[ticker, 'Date'])
        X_test = X_test.iloc[0]
        y_test = test_df[ticker]

        garch_model = arch_model(y_train, vol='Garch', p=1, q=1)
        garch_results = garch_model.fit()

        forecasts = garch_results.forecast(start=0, horizon=9)  # Forecast for 9 months ahead
        cond_vol_forecast = forecasts.variance.iloc[-1].apply(lambda x: np.sqrt(x))
        mean_forecast = forecasts.mean.iloc[-1]

        VaR_forecast = mean_forecast + cond_vol_forecast * norm.ppf(alpha)

        loss = tick_loss(alpha, y_test.iloc[0], VaR_forecast)

        t_loss += loss

    t_loss = t_loss / 4  # Adjusted for 4 months (since 'dates9m' has 4 elements)

    return t_loss

In [22]:
dates9m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01']

In [23]:
for t in tickers:
    loss = rolling_9m_forecast(data, dates9m, t, 0.05)
    print(f'{t}: {loss}')

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1212.2397444537628
Iteration:      2,   Func. Count:     13,   Neg. LLF: 878.8800633143923
Iteration:      3,   Func. Count:     19,   Neg. LLF: 876.5126927333897
Iteration:      4,   Func. Count:     26,   Neg. LLF: 874.6973681636873
Iteration:      5,   Func. Count:     31,   Neg. LLF: 874.6903544449096
Iteration:      6,   Func. Count:     36,   Neg. LLF: 874.6560717780487
Iteration:      7,   Func. Count:     41,   Neg. LLF: 874.6348053303011
Iteration:      8,   Func. Count:     46,   Neg. LLF: 874.6216452705775
Iteration:      9,   Func. Count:     51,   Neg. LLF: 874.6199930891166
Iteration:     10,   Func. Count:     56,   Neg. LLF: 874.6199821054033
Iteration:     11,   Func. Count:     60,   Neg. LLF: 874.6199821052801
Optimization terminated successfully    (Exit mode 0)
            Current function value: 874.6199821054033
            Iterations: 11
            Function evaluations: 60
            Gradient evaluations: 1

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1226.3396128307274
Iteration:      2,   Func. Count:     13,   Neg. LLF: 887.6477778013036
Iteration:      3,   Func. Count:     19,   Neg. LLF: 898.7967089181791
Iteration:      4,   Func. Count:     26,   Neg. LLF: 883.5483377897249
Iteration:      5,   Func. Count:     31,   Neg. LLF: 883.5428403701119
Iteration:      6,   Func. Count:     36,   Neg. LLF: 883.5134750240936
Iteration:      7,   Func. Count:     41,   Neg. LLF: 883.4874299759567
Iteration:      8,   Func. Count:     46,   Neg. LLF: 883.4737747768788
Iteration:      9,   Func. Count:     51,   Neg. LLF: 883.4696451596949
Iteration:     10,   Func. Count:     56,   Neg. LLF: 883.4696170540337
Iteration:     11,   Func. Count:     60,   Neg. LLF: 883.4696170542519
Optimization terminated successfully    (Exit mode 0)
            Current function value: 883.4696170540337
            Iterations: 11
            Function evaluations: 60
            Gradient evaluations: 1

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 916.6039256453187
Iteration:      2,   Func. Count:     13,   Neg. LLF: 918.8551304785537
Iteration:      3,   Func. Count:     19,   Neg. LLF: 916.9226917484118
Iteration:      4,   Func. Count:     26,   Neg. LLF: 897.5640206485998
Iteration:      5,   Func. Count:     31,   Neg. LLF: 897.5634798552542
Iteration:      6,   Func. Count:     36,   Neg. LLF: 897.5633757100527
Iteration:      7,   Func. Count:     41,   Neg. LLF: 897.5627197625347
Iteration:      8,   Func. Count:     46,   Neg. LLF: 897.559328176062
Iteration:      9,   Func. Count:     51,   Neg. LLF: 897.5478936577738
Iteration:     10,   Func. Count:     56,   Neg. LLF: 897.5455189506919
Iteration:     11,   Func. Count:     61,   Neg. LLF: 897.5450656952354
Iteration:     12,   Func. Count:     66,   Neg. LLF: 897.544964183797
Iteration:     13,   Func. Count:     71,   Neg. LLF: 897.5449634608681
Optimization terminated successfully    (Exit mode 0)
            

...

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

NVS: 1.3727909062970607
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1062.9352105492499
Iteration:      2,   Func. Count:     14,   Neg. LLF: 855.6678210371897
Iteration:      3,   Func. Count:     21,   Neg. LLF: 863.0449350253768
Iteration:      4,   Func. Count:     28,   Neg. LLF: 839.8714119552811
Iteration:      5,   Func. Count:     33,   Neg. LLF: 839.8083805174045
Iteration:      6,   Func. Count:     38,   Neg. LLF: 841.1909442545641
Iteration:      7,   Func. Count:     44,   Neg. LLF: 839.7968870734759
Iteration:      8,   Func. Count:     50,   Neg. LLF: 839.7717857611278
Iteration:      9,   Func. Count:     55,   Neg. LLF: 839.77171073988
Iteration:     10,   Func. Count:     59,   Neg. LLF: 839.7717107398021
Optimization terminated successfully    (Exit mode 0)
            Current function value: 839.77171073988
            Iterations: 10
            Function evaluations: 59
            Gradient evaluations: 10
Iteration:      1,   Func. Count:      6,   Neg. L

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 861.8820674751669
Iteration:      2,   Func. Count:     13,   Neg. LLF: 973.4386749461528
Iteration:      3,   Func. Count:     20,   Neg. LLF: 11746.250322742857
Iteration:      4,   Func. Count:     28,   Neg. LLF: 849.9250194096192
Iteration:      5,   Func. Count:     33,   Neg. LLF: 849.8758198813453
Iteration:      6,   Func. Count:     38,   Neg. LLF: 850.6206074513934
Iteration:      7,   Func. Count:     44,   Neg. LLF: 849.8669093131068
Iteration:      8,   Func. Count:     50,   Neg. LLF: 849.8425439148643
Iteration:      9,   Func. Count:     55,   Neg. LLF: 849.8424541658421
Iteration:     10,   Func. Count:     59,   Neg. LLF: 849.8424541656949
Optimization terminated successfully    (Exit mode 0)
            Current function value: 849.8424541658421
            Iterations: 10
            Function evaluations: 59
            Gradient evaluations: 10
PFE: 3.3883448501887212
Iteration:      1,   Func. Count:      6,   Ne

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

# 12 MONTHS AHEAD:

In [24]:
import pandas as pd
import statsmodels.api as sm
from arch import arch_model
from scipy.stats import norm

def rolling_12m_forecast(data, dates, ticker, alpha):
    t_loss = 0
    dta = data.copy()

    for d in dates:
        train_df = dta[dta.Date <= d]
        test_df = dta[dta.Date > d]
        test_df = test_df.reset_index(drop=True)

        X_train = train_df.drop(columns=[ticker, 'Date'])
        y_train = train_df[ticker]

        X_test = test_df.drop(columns=[ticker, 'Date'])
        X_test = X_test.iloc[0]
        y_test = test_df[ticker]

        garch_model = arch_model(y_train, vol='Garch', p=1, q=1)
        garch_results = garch_model.fit()

        forecasts = garch_results.forecast(start=0, horizon=12)  # Forecast for 12 months ahead
        cond_vol_forecast = forecasts.variance.iloc[-1].apply(lambda x: np.sqrt(x))
        mean_forecast = forecasts.mean.iloc[-1]

        VaR_forecast = mean_forecast + cond_vol_forecast * norm.ppf(alpha)

        loss = tick_loss(alpha, y_test.iloc[0], VaR_forecast)

        t_loss += loss

    t_loss = t_loss / len(dates)  # Adjusted for the number of months in 'dates12m'

    return t_loss

In [25]:
dates12m = ['2022-05-01']

In [26]:
for t in tickers:
    loss = rolling_12m_forecast(data, dates12m, t, 0.05)
    print(f'{t}: {loss}')

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1212.2397444537628
Iteration:      2,   Func. Count:     13,   Neg. LLF: 878.8800633143923
Iteration:      3,   Func. Count:     19,   Neg. LLF: 876.5126927333897
Iteration:      4,   Func. Count:     26,   Neg. LLF: 874.6973681636873
Iteration:      5,   Func. Count:     31,   Neg. LLF: 874.6903544449096
Iteration:      6,   Func. Count:     36,   Neg. LLF: 874.6560717780487
Iteration:      7,   Func. Count:     41,   Neg. LLF: 874.6348053303011
Iteration:      8,   Func. Count:     46,   Neg. LLF: 874.6216452705775
Iteration:      9,   Func. Count:     51,   Neg. LLF: 874.6199930891166
Iteration:     10,   Func. Count:     56,   Neg. LLF: 874.6199821054033
Iteration:     11,   Func. Count:     60,   Neg. LLF: 874.6199821052801
Optimization terminated successfully    (Exit mode 0)
            Current function value: 874.6199821054033
            Iterations: 11
            Function evaluations: 60
            Gradient evaluations: 1

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


BMY: 7.618469905641936
Iteration:      1,   Func. Count:      6,   Neg. LLF: 138543517.9806407
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1021.5378549963325
Iteration:      3,   Func. Count:     19,   Neg. LLF: 844.9563210056439
Iteration:      4,   Func. Count:     25,   Neg. LLF: 781.4032102405062
Iteration:      5,   Func. Count:     31,   Neg. LLF: 785.1828167565667
Iteration:      6,   Func. Count:     37,   Neg. LLF: 779.6972234375635
Iteration:      7,   Func. Count:     43,   Neg. LLF: 846.0873417460587
Iteration:      8,   Func. Count:     49,   Neg. LLF: 775.2467209166093
Iteration:      9,   Func. Count:     55,   Neg. LLF: 773.6211539806227
Iteration:     10,   Func. Count:     61,   Neg. LLF: 773.6003032018345
Iteration:     11,   Func. Count:     66,   Neg. LLF: 773.5998619019424
Iteration:     12,   Func. Count:     71,   Neg. LLF: 773.5998591684158
Iteration:     13,   Func. Count:     75,   Neg. LLF: 773.5998591692786
Optimization terminated successfully    

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


LLY: 8.818029204888099
Iteration:      1,   Func. Count:      6,   Neg. LLF: 979.0685822154014
Iteration:      2,   Func. Count:     12,   Neg. LLF: 129676.49104452106
Iteration:      3,   Func. Count:     20,   Neg. LLF: 887.0274896157559
Iteration:      4,   Func. Count:     26,   Neg. LLF: 889.1553343201592
Iteration:      5,   Func. Count:     33,   Neg. LLF: 875.5267809032181
Iteration:      6,   Func. Count:     39,   Neg. LLF: 874.8990213523917
Iteration:      7,   Func. Count:     45,   Neg. LLF: 874.5955077209251
Iteration:      8,   Func. Count:     50,   Neg. LLF: 874.5908127513169
Iteration:      9,   Func. Count:     55,   Neg. LLF: 874.5906659407244
Iteration:     10,   Func. Count:     60,   Neg. LLF: 874.5906402867531
Iteration:     11,   Func. Count:     64,   Neg. LLF: 874.5906402863982
Optimization terminated successfully    (Exit mode 0)
            Current function value: 874.5906402867531
            Iterations: 11
            Function evaluations: 64
            

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

NVS: 0.0599190777167657
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1062.9352105492499
Iteration:      2,   Func. Count:     14,   Neg. LLF: 855.6678210371897
Iteration:      3,   Func. Count:     21,   Neg. LLF: 863.0449350253768
Iteration:      4,   Func. Count:     28,   Neg. LLF: 839.8714119552811
Iteration:      5,   Func. Count:     33,   Neg. LLF: 839.8083805174045
Iteration:      6,   Func. Count:     38,   Neg. LLF: 841.1909442545641
Iteration:      7,   Func. Count:     44,   Neg. LLF: 839.7968870734759
Iteration:      8,   Func. Count:     50,   Neg. LLF: 839.7717857611278
Iteration:      9,   Func. Count:     55,   Neg. LLF: 839.77171073988
Iteration:     10,   Func. Count:     59,   Neg. LLF: 839.7717107398021
Optimization terminated successfully    (Exit mode 0)
            Current function value: 839.77171073988
            Iterations: 10
            Function evaluations: 59
            Gradient evaluations: 10
PFE: 7.277013376509613
Iteration:      1,   Func. 

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
